To install OR-Tools, run the following cell:

In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.
tensorflow-metadata 1.13.1 requires absl-py<2.0.0,>=0.9, but you have absl-py 2.1.0 which is incompatible

You are given several classes for reading the instance data from a file, storing the instance data and returning a solution, as well as some testing facilities:

In [2]:
class AuthorisationConstraint:
    def __init__(self, instance, user, tasks):
        assert user >= 0 and user < instance.m
        assert all(t >= 0 and t < instance.n for t in tasks)

        self.user = user
        self.tasks = tasks

    def is_satisfied(self, solution):
        for task in set(range(solution.instance.n)) - set(self.tasks):
            if solution.assignment[task] == self.user:
                return False
        return True

    def write(self, f):
        f.write('Authorisations u' + str(self.user + 1))
        for t in self.tasks:
            f.write(' t' + str(t + 1))
        f.write('\n')


class BindingOfDutyConstraint:
    def __init__(self, instance, t1, t2):
        assert 0 <= t1 < instance.n
        assert t2 >= 0 and t2 < instance.n

        self.t1 = t1
        self.t2 = t2

    def is_satisfied(self, solution):
        return solution.assignment[self.t1] == solution.assignment[self.t2]

    def write(self, f):
        f.write('Binding-of-duty t%i t%i\n' % (self.t1 + 1, self.t2 + 1))


class SeparationOfDutyConstraint:
    def __init__(self, instance, t1, t2):
        assert 0 <= t1 < instance.n
        assert t2 >= 0 and t2 < instance.n

        self.t1 = t1
        self.t2 = t2

    def is_satisfied(self, solution):
        return solution.assignment[self.t1] != solution.assignment[self.t2]

    def write(self, f):
        f.write('Separation-of-duty t%i t%i\n' % (self.t1 + 1, self.t2 + 1))


class AtMostKConstraint:
    def __init__(self, instance, k, tasks):
        assert 0 < k <= instance.n
        assert all(0 <= task <= instance.n for task in tasks)

        self.tasks = tasks
        self.k = k

    def is_satisfied(self, solution):
        return len(set(solution.assignment[task] for task in self.tasks)) <= self.k

    def write(self, f):
        f.write('At-most-k %i' % self.k)
        for t in self.tasks:
            f.write(' t%i' % (t + 1))
        f.write('\n')


class ExtensionConstraint1:
    def __init__(self, instance, k, tasks):
        assert 1 <= k <= instance.n
        assert all(0 <= t < instance.n for t in tasks)

        self.tasks = tasks
        self.k = k

    def is_satisfied(self, solution):
        return len(set(solution.assignment[t] for t in self.tasks)) == self.k

    def write(self, f):
        f.write(f'EC1 {self.k} {" ".join(f"t{t+1}" for t in self.tasks)}\n')


class ExtensionConstraint2:
    def __init__(self, instance, tasks, teams):
        assert all(0 <= t < instance.n for t in tasks)
        assert all(0 <= team < len(instance.teams) for team in teams)

        self.tasks = tasks
        self.teams = teams
        self.instance = instance

    def is_satisfied(self, solution):
        return any(all(solution.assignment[t] in self.instance.teams[team] for t in self.tasks) for team in self.teams)

    def write(self, f):
        f.write(f'EC2 {" ".join(f"team{team+1}" for team in self.teams)} {" ".join(f"t{t+1}" for t in self.tasks)}\n')


class ExtensionConstraint3:
    def __init__(self, instance, team, k):
        assert 0 <= k < instance.n
        assert 0 <= team < len(instance.teams)

        self.team = team
        self.k = k
        self.instance = instance

    def is_satisfied(self, solution):
        return sum(1 for t in range(self.instance.n) if solution.assignment[t] in self.instance.teams[self.team]) <= self.k

    def write(self, f):
        f.write(f'EC3 team{self.team + 1} {self.k}\n')


class ExtensionConstraint4:
    def __init__(self, instance, team, supervisor):
        assert 0 <= team < len(instance.teams)
        assert 0 <= supervisor < instance.m

        self.team = team
        self.supervisor = supervisor
        self.instance = instance

    def is_satisfied(self, solution):
        return not any(solution.assignment[t] in self.instance.teams[self.team] for t in range(self.instance.n)) \
            or any(solution.assignment[t] == self.supervisor for t in range(self.instance.n))

    def write(self, f):
        f.write(f'EC4 team{self.team + 1} u{self.supervisor + 1}\n')


# Reads and stores instance data
class Instance:
    def __init__(self, filename):
        def parse_task(string):
            return int(re.match(r't(\d+)', string).group(1)) - 1

        def parse_user(string):
            return int(re.match(r'u(\d+)', string).group(1)) - 1

        def parse_team(string):
            return int(re.match(r'team(\d+)', string).group(1)) - 1

        if filename is None:
            return

        with open(filename, 'r') as f:
            import re
            self.n = int(re.match(r'^\s*#Tasks:\s+(\d+)\s*$', f.readline(), re.IGNORECASE).group(1))
            self.m = int(re.match(r'^\s*#Users:\s+(\d+)\s*$', f.readline(), re.IGNORECASE).group(1))

            t = int(re.match(r'^\s*#Teams:\s+(\d+)\s*$', f.readline(), re.IGNORECASE).group(1))
            self.teams = []
            for team_index in range(t):
                self.teams.append(list(map(parse_user, f.readline().strip().lower().split())))

            c = int(re.match(r'^\s*#Constraints:\s+(\d+)\s*$', f.readline(), re.IGNORECASE).group(1))

            self.constraints = []
            for line_index in range(c):
                line = f.readline().strip().lower()
                values = line.split()

                if values[0] == 'authorisations':
                    self.constraints.append(AuthorisationConstraint(
                        self, parse_user(values[1]), list(map(parse_task, values[2:]))))

                elif values[0] == 'binding-of-duty':
                    self.constraints.append(BindingOfDutyConstraint(self, parse_task(values[1]), parse_task(values[2])))

                elif values[0] == 'separation-of-duty':
                    self.constraints.append(SeparationOfDutyConstraint(self, parse_task(values[1]), parse_task(values[2])))

                elif values[0] == 'at-most-k':
                    self.constraints.append(AtMostKConstraint(self, int(values[1]), list(map(parse_task, values[2:]))))

                elif values[0] == 'ec1':
                    self.constraints.append(ExtensionConstraint1(self, int(values[1]), list(map(parse_task, values[2:]))))

                elif values[0] == 'ec2':
                    teams = []
                    index = 1
                    while values[index].startswith('team'):
                        teams.append(parse_team(values[index]))
                        index += 1

                    self.constraints.append(ExtensionConstraint2(self, list(map(parse_task, values[index:])), teams))

                elif values[0] == 'ec3':
                    self.constraints.append(ExtensionConstraint3(self, parse_team(values[1]), int(values[2])))

                elif values[0] == 'ec4':
                    self.constraints.append(ExtensionConstraint4(self, parse_team(values[1]), parse_user(values[2])))

                else:
                    raise Exception(f'Unknown constraint {values[0]}.')

    def save(self, filename):
        import os, sys
        with open(filename, 'w') as f:
            f.write('#Tasks: ' + str(self.n) + '\n')
            f.write('#Users: ' + str(self.m) + '\n')
            f.write('#Teams: ' + str(len(self.teams)) + '\n')

            for team in self.teams:
                f.write(' '.join(f'u{u+1}' for u in team))
                f.write('\n')

            f.write('#Constraints: ' + str(len(self.constraints)) + '\n')
            for c in self.constraints:
                c.write(f)


# Stores a solution to a WSP instance
class Solution:
    def __init__(self, instance, sat):
        self.instance = instance
        self.sat = sat
        self.assignment = [-1]*self.instance.n

    # Use this function to specify that user 'user' is assigned to task 'task'
    def assign_user(self, task, user):
        if task < 0 or task >= self.instance.n:
            raise Exception(f'Task {task} is outside the range.')

        if user < 0 or user >= self.instance.m:
            raise Exception(f'User {user} is outside the range.')

        self.assignment[task] = user


def ensure_instances_downloaded():
    from os.path import exists

    if not exists('instances.zip'):
        print(f'Downloading \'instances.zip\'...')

        url = 'https://www.dropbox.com/scl/fi/vnq9okzulpkcs43nvc4m1/project2instances.zip?rlkey=2c4ewpi4kik4bpflzqtzg0xx9&dl=1'

        import urllib
        req = urllib.request.Request(url)

        with urllib.request.urlopen(req) as file:
            with open('instances.zip', 'wb') as f:
                f.write(file.read())

        print(f'Unpacking...')
        import zipfile
        with zipfile.ZipFile('instances.zip', 'r') as zip_ref:
            zip_ref.extractall('.')

        print(f'Instances are ready')


def run_test(filename, known_to_be_sat):
    def coloured_print(text, colour):
        from IPython.core.display import display, HTML
        display(HTML(f'<span style=color:{colour}><pre>{text}</pre></span>'))

    ensure_instances_downloaded()

    instance = Instance(filename)

    import time

    starttime = time.perf_counter()
    solution = solve(instance)
    endtime = time.perf_counter()

    def print_test_header(passed):
        coloured_print(f'{filename:<30} {"  (sat)" if known_to_be_sat else "(unsat)"} {(endtime - starttime) * 1000:5.0f} ms {"PASS" if passed else "FAIL"}', 'green' if passed else 'red')

    if solution.sat:
        if len(solution.assignment) != instance.n or not all(0 <= user <= instance.m for user in solution.assignment):
            print_test_header(False)
            print('  assignment of users to tasks is infeasible.')
            return float('inf')

        broken = [c for c in instance.constraints if not c.is_satisfied(solution)]
        if len(broken) > 0:
            print_test_header(False)
            print('  the solution breaks some constraints:')
            for t in [AuthorisationConstraint, BindingOfDutyConstraint, SeparationOfDutyConstraint, AtMostKConstraint]:
                print(f'    {len([0 for c in broken if isinstance(c, t)])} broken {t.__name__} constraints')
            return float('inf')

    correct = solution.sat == known_to_be_sat
    if correct:
        print_test_header(True)
    else:
        print_test_header(False)
        print(f'  Expected outcome:  {"sat" if known_to_be_sat else "unsat"}')
        print(f'    Actual outcome:  {"sat" if solution.sat else "unsat"}')

    return endtime - starttime


ensure_instances_downloaded()


def test_batch(batch_index: int):
    print(f'Testing batch{batch_index}:')
    total = 0.0
    for i in range(1, 11):
        total += run_test(f'batch{batch_index}/inst_{i}.txt', i % 2 == 1)

    print(f'Total time for batch{batch_index}: {total:0.1f} sec\n')


Unpacking...
Instances are ready


You are expected to implement function `solve(instance)` that takes an object of class `Instance` as a parameter and returns an object of class `Solution`.

You can extend the functionality of the provided classes as you wish.  If you change those classes, please include them into your submission.

Implement the `solve(instance)` function below.

In [3]:
from ortools.sat.python import cp_model
from itertools import combinations

def solve(instance):

    model = cp_model.CpModel()

    A = [model.NewIntVar(0, instance.m - 1, '') for i in range(instance.n)]
    M = [[model.NewBoolVar('') for j in range(instance.n)] for i in range(instance.n)]

    for t1, t2 in combinations(range(instance.n), 2):
        model.Add(M[t1][t2] == M[t2][t1])

    for t in range(instance.n):
        model.Add(M[t][t] == 1)

    for t1 in range(instance.n - 2):
        for t2 in range(t1 + 1, instance.n - 1):
            for t3 in range(t2 + 1, instance.n):
                model.Add(M[t1][t3] == 1).OnlyEnforceIf([M[t1][t2], M[t2][t3]])
                model.Add(M[t1][t3] == 0).OnlyEnforceIf([M[t1][t2].Not(), M[t2][t3]])
                model.Add(M[t1][t3] == 0).OnlyEnforceIf([M[t1][t2], M[t2][t3].Not()])

    for t1, t2 in combinations(range(instance.n), 2):
        model.Add(A[t1] == A[t2]).OnlyEnforceIf(M[t1][t2])
        model.Add(A[t1] != A[t2]).OnlyEnforceIf(M[t1][t2].Not())


    for constraint in instance.constraints:
        if isinstance(constraint, AuthorisationConstraint):
            for t in (set(range(instance.n)) - set(constraint.tasks)):
                model.Add(A[t] != constraint.user)

        elif isinstance(constraint, BindingOfDutyConstraint):
            model.Add(M[constraint.t1][constraint.t2]==1)

        elif isinstance(constraint, SeparationOfDutyConstraint):
            model.Add(M[constraint.t1][constraint.t2]==0)

        elif isinstance(constraint, AtMostKConstraint):
            for subset in combinations(constraint.tasks, constraint.k + 1):
                model.AddBoolOr([M[t1][t2] for t1, t2 in combinations(subset, 2)])

        elif isinstance(constraint, ExtensionConstraint1):
            for subset in combinations(constraint.tasks, constraint.k + 1):
                model.AddBoolOr([M[s1][s2] for s1, s2 in combinations(subset, 2)])

            bools = []
            for subset in combinations(constraint.tasks, constraint.k):
                b = model.NewBoolVar('')
                model.AddBoolAnd([M[t1][t2].Not() for t1, t2 in combinations(subset, 2)]).OnlyEnforceIf(b)
                bools.append(b)
            model.AddBoolOr(bools)


        elif isinstance(constraint, ExtensionConstraint2):
            bools = []
            for team in constraint.teams:
                b = model.NewBoolVar('')
                x = set(range(instance.m)) - set(instance.teams[team])
                for t in constraint.tasks:
                    for u in x:
                        model.Add(A[t] != u).OnlyEnforceIf(b)
                bools.append(b)
            model.Add(sum(bools) == 1)


        elif isinstance(constraint, ExtensionConstraint3):
            bools = []
            for t in range(instance.n):
                b = model.NewBoolVar('')
                for u in instance.teams[constraint.team]:
                    model.Add(A[t] != u).OnlyEnforceIf(b.Not())
                bools.append(b)
            model.Add(sum(bools) <= constraint.k)


        elif isinstance(constraint, ExtensionConstraint4):

            T = model.NewBoolVar('')
            bools = []
            for t in range(instance.n):
                b = model.NewBoolVar('')
                for u in instance.teams[constraint.team]:
                    model.Add(A[t] != u).OnlyEnforceIf(b.Not())
                model.Add(T == 1).OnlyEnforceIf(b)
                bools.append(b)
            model.Add(T == 0).OnlyEnforceIf([b.Not() for b in bools])

            S = model.NewBoolVar('')
            bools = []
            for t in range(instance.n):
                b = model.NewBoolVar('')
                model.Add(A[t] == constraint.supervisor).OnlyEnforceIf(b)
                model.Add(S == 1).OnlyEnforceIf(b)
                bools.append(b)
            model.Add(S == 0).OnlyEnforceIf([b.Not() for b in bools])

            model.Add(S == 1).OnlyEnforceIf(T)


    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    solution = Solution(instance, status in [cp_model.FEASIBLE, cp_model.OPTIMAL])
    if solution.sat:
        for i in range(instance.n):
            solution.assign_user(i, solver.Value(A[i]))
        print(solution.assignment)

    return solution





def solve1(instance):

    model = cp_model.CpModel()

    A = [[model.NewBoolVar('') for i in range(instance.m)] for i in range(instance.n)]
    M = [[model.NewBoolVar('') for j in range(instance.n)] for i in range(instance.n)]

    for task in range(instance.n):
        model.Add(sum(A[task]) == 1)

    for t1 in range(instance.n - 1):
        for t2 in range(t1 + 1, instance.n):
            model.Add(M[t1][t2] == M[t2][t1])

    for t in range(instance.n):
        model.Add(M[t][t] == 1)

    for t1 in range(instance.n - 2):
        for t2 in range(t1 + 1, instance.n - 1):
            for t3 in range(t2 + 1, instance.n):
                model.Add(M[t1][t3] == 1).OnlyEnforceIf([M[t1][t2], M[t2][t3]])
                model.Add(M[t1][t3] == 0).OnlyEnforceIf([M[t1][t2].Not(), M[t2][t3]])
                model.Add(M[t1][t3] == 0).OnlyEnforceIf([M[t1][t2], M[t2][t3].Not()])

    for t1 in range(instance.n - 1):
        for t2 in range(t1 + 1, instance.n):
            for u in range(instance.m):
                model.Add(A[t1][u] == A[t2][u]).OnlyEnforceIf(M[t1][t2])
                model.AddBoolOr([A[t1][u].Not(), A[t2][u].Not()]).OnlyEnforceIf(M[t1][t2].Not())

    for constraint in instance.constraints:
        if isinstance(constraint, AuthorisationConstraint):
            for t in (set(range(instance.n)) - set(constraint.tasks)):
                model.Add(A[t][constraint.user] == 0)

        elif isinstance(constraint, BindingOfDutyConstraint):
            model.Add(M[constraint.t1][constraint.t2] == 1)

        elif isinstance(constraint, SeparationOfDutyConstraint):
            model.Add(M[constraint.t1][constraint.t2] == 0)

        elif isinstance(constraint, AtMostKConstraint):
            for subset in combinations(constraint.tasks, constraint.k + 1):
                model.AddBoolOr([M[s1][s2] for s1, s2 in combinations(subset, 2)])

        elif isinstance(constraint, ExtensionConstraint1):
            p = {}
            for t in constraint.tasks:
                p[t] = model.NewIntVar(0, constraint.k - 1, '')

            for t1, t2 in combinations(constraint.tasks, 2):
                model.Add(p[t1] == p[t2]).OnlyEnforceIf(M[t1][t2])
                model.Add(p[t1] != p[t2]).OnlyEnforceIf(M[t1][t2].Not())

            for i in range(constraint.k):
                bools = []
                for t in constraint.tasks:
                    b = model.NewBoolVar('')
                    model.Add(p[t] == i).OnlyEnforceIf(b)
                    bools.append(b)
                model.AddBoolOr(bools)

        elif isinstance(constraint, ExtensionConstraint2):
            bools = []
            for team in constraint.teams:
                b = model.NewBoolVar('')
                for t in constraint.tasks:
                    model.AddBoolOr([A[t][u] for u in instance.teams[team]]).OnlyEnforceIf(b)
                bools.append(b)
            model.Add(sum(bools) == 1)


        elif isinstance(constraint, ExtensionConstraint3):
            bools = []
            for t in range(instance.n):
                b = model.NewBoolVar('')
                model.AddBoolAnd([A[t][u].Not() for u in instance.teams[constraint.team]]).OnlyEnforceIf(b.Not())
                bools.append(b)
            model.Add(sum(bools) <= constraint.k)


        elif isinstance(constraint, ExtensionConstraint4):

            T = model.NewBoolVar('')
            bools = []
            for t in range(instance.n):
                b = model.NewBoolVar('')
                for u in instance.teams[constraint.team]:
                    model.Add(A[t][u] == 0).OnlyEnforceIf(b.Not())
                model.Add(T == 1).OnlyEnforceIf(b)
                bools.append(b)
            model.Add(T == 0).OnlyEnforceIf([b.Not() for b in bools])

            S = model.NewBoolVar('')
            bools = []
            for t in range(instance.n):
                b = model.NewBoolVar('')
                model.Add(A[t][constraint.supervisor] == 1).OnlyEnforceIf(b)
                model.Add(S == 1).OnlyEnforceIf(b)
                bools.append(b)
            model.Add(S == 0).OnlyEnforceIf([b.Not() for b in bools])

            model.Add(S == 1).OnlyEnforceIf(T)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    solution = Solution(instance, status in [cp_model.FEASIBLE, cp_model.OPTIMAL])
    if solution.sat:
        for i in range(instance.n):
            for j in range(instance.m):
                if solver.Value(A[i][j]):
                    solution.assign_user(i, j)
        print(solution.assignment)

    return solution




Run this cell to test your `solve(instance)` function.

In [6]:
"""
for batch_index in range(7):
    test_batch(batch_index)
"""
#test_batch(0)
#test_batch(1)
#test_batch(2)
#test_batch(3)
#test_batch(4)
#test_batch(5)
#test_batch(6)


Testing batch0:
[6, 4, 1, 1, 2, 4, 5, 3, 5, 3, 1, 4]


[8, 4, 5, 3, 10, 2, 9, 9, 0, 1, 7, 0, 5, 6, 2, 2]


[2, 3, 3, 7, 4, 0, 4, 1, 8, 1, 6, 0, 3, 0, 0, 1, 2, 0, 2, 1]


[13, 18, 9, 1, 9, 10, 3, 6, 5, 6, 6, 1, 3, 8, 18, 2, 2, 2, 4, 4, 2, 4, 0, 0]


[11, 3, 6, 4, 5, 3, 1, 6, 0, 4, 0, 2, 0, 1, 0, 7, 1, 2, 0, 4, 3, 6, 4, 2, 2, 4, 11, 5]


Total time for batch0: 4.1 sec

